In [0]:
import tensorflow as tf
import os
import numpy as np

In [2]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

Cloning into 'fenwicks'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 743 (delta 73), reused 102 (delta 38), pack-reused 600
Receiving objects: 100% (743/743), 1.34 MiB | 4.92 MiB/s, done.
Resolving deltas: 100% (399/399), done.


In [3]:
BUCKET = 'gs://gs_colab'
PROJECT = 'tf_flowers'
MODEL = "ResNet50" #@param ["InceptionResNetV2", "ResNet50", "ResNet50V2", "InceptionV3", "MobileNetV2", "Xception"]

BATCH_SIZE = 128 #@param ["64", "128", "256", "512"] {type:"raw"}
EPOCHS = 16 #@param {type:"slider", min:1, max:100, step:1}


In [4]:
fw.colab_utils.setup_gcs()

In [5]:
data_dir, work_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT)

In [6]:
base_model = fw.keras_models.get_model(MODEL, BUCKET)

In [7]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.FLOWER_PHOTOS, './flower_photos')

228818944/228813984 [==============================] - 2s 0us/step


In [8]:
data_dir_local = os.path.join(data_dir_local, 'flower_photos')
data_dir_local

'./flower_photos/datasets/flower_photos'

In [9]:
data_fn = os.path.join(data_dir, 'all.tfrecord')

paths, y, labels = fw.io.data_dir_tfrecord(data_dir_local, data_fn, shuffle=True)
n_classes = len(labels)
n_all = len(y)

In [10]:
def build_nn():
  base = base_model.model_func()
  base.trainable = False
  model = fw.Sequential()
  model.add(base)
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(n_classes, use_bias=False))
  return model

In [11]:
n_valid = n_all // 5 // 8 * 8
n_train = n_all - n_valid
total_steps = n_train // BATCH_SIZE * EPOCHS

In [12]:
lr_func = fw.train.cosine_lr(init_lr=0.001, total_steps=total_steps)
fw.plt.plot_lr_func(lr_func, total_steps)

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
opt_func = fw.train.adam_optimizer(lr_func)
model_func = fw.tpuest.get_clf_model_func(build_nn, opt_func)

In [14]:
parser_train = fw.io.get_tfexample_image_parser(base_model.img_size, base_model.img_size, training=True, normalizer=base_model.normalizer)
parser_eval = fw.io.get_tfexample_image_parser(base_model.img_size, base_model.img_size, training=False, normalizer=base_model.normalizer)


est = fw.tpuest.get_tpu_estimator(n_train, n_valid, model_func, work_dir, 
                             base_model.weight_dir, base_model.weight_vars, BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/tf_flowers/2019-04-19-19:53:11', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.30.185.170:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb2698a5080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.30.185.170:8470', '_evaluation_master': 'grpc://10.30.185.170:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=22, num_shards=

In [15]:
def train_eval_fold(val_fold):
  train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
    params['batch_size'], n_folds=5, val_fold_idx = val_fold, training=True)
  valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
    params['batch_size'], n_folds=5, val_fold_idx = val_fold, training=False)

  est.train(train_input_func, steps=total_steps)
  result = est.evaluate(valid_input_func, steps=1)
  fw.io.create_clean_dir(work_dir)
  return result

In [16]:
result = []
result.append(train_eval_fold(0))

INFO:tensorflow:Querying Tensorflow master (grpc://10.30.185.170:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 15921547395312216294)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 15460531492973742864)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6588000549386914446)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3635027615589392836)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 265459006814856477)
INFO:tensorflow:*** Available Device: _DeviceAttributes(

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[

In [17]:
result.append(train_eval_fold(1))

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [18]:
result.append(train_eval_fold(2))

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [19]:
result.append(train_eval_fold(3))

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [20]:
result.append(train_eval_fold(4))

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [21]:
acc = [res['accuracy'] for res in result]
loss = [res['loss'] for res in result]

print('accuracy:', np.mean(acc), '+/-', np.std(acc))
print('loss:', np.mean(loss), '+/-', np.std(loss))

accuracy: 0.90934056 +/- 0.007473334
loss: 0.21930814 +/- 0.011603798
